In [2]:
import pandas as pd
import numpy as np
import torch
from pytorch_pretrained_bert import BertTokenizer,BertModel
import logging
import difflib

`data_withGoldenLabel`为人工标注标签

`data_withSilverLabel`为算法标注标签

`data`去重:重复数据直接删除

将`author`中`nan`替换为`'Not Available'`

In [23]:
data = pd.read_table("./DataSet/book/book.txt" , sep='\t' , header=None , names=['source','isbn','name','author'])
data = data.drop_duplicates().reset_index(drop=True)
data.fillna('Not Available', inplace=True)
data['encode']=None
GoldenLabel = pd.read_table("./DataSet/book/book_golden.txt" , sep='\t' , header=None , names=['isbn','author'])
SilverLabel = pd.read_table("./DataSet/book/book_silver.txt" , sep='\t' , header=None , names=['isbn','author'])
SilverLabel = SilverLabel[SilverLabel['isbn']!='F']

加载`bert`模型和分词器

In [10]:
logging.basicConfig(level=logging.INFO)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at C:\Users\zxr\.pytorch_pretrained_bert\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at C:\Users\zxr\.pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file C:\Users\zxr\.pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir C:\Users\zxr\AppData\Local\Temp\tmp770fcb27
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_pr

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

逐条处理数据

选用`[CLS]`的最后一个隐层编码

GPU内存不足，暂定在CPU上运行

In [27]:
for i in range(0,len(data)):
    item = data.loc[i]
    text = '[CLS] ' + (item['source']) + ' [SEP] ' + (item['name']) + ' [SEP] ' + (item['author']) + ' [SEP]'
    if (i%100==0):
        print(i,text)
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    #torch.cuda.empty_cache()
    #model.to('cuda')
    #tokens_tensor = tokens_tensor.to('cuda')
    with torch.no_grad():
        encoded_layers,_ = model(tokens_tensor)
    data.loc[i]['encode'] = encoded_layers[11][0][0]

0 [CLS] eCampus.com [SEP] The art Of Computer Programming, Fascicle 3: Generating All Combinations And Partitions [SEP] Not Available [SEP]
100 [CLS] Revaluation Books [SEP] The Art of Computer Programming, Volume 1, Fascicle 1: MMIX -- A RISC Computer for the New Millennium (Art of Computer Programming) [SEP] Donald E. Knuth [SEP]
200 [CLS] Marando.de Versandbuchhandlung [SEP] Algorithms in C, 2 vols., Engl. ed. [SEP] Robert Sedgewick [SEP]
300 [CLS] The E [SEP] The Art of Computer Programming: Fundamental Algorithms [SEP] Knuth, Donald E.; Knuth, Donald [SEP]
400 [CLS] Livrenoir [SEP] An Introduction to Parallel Algorithms [SEP] Jaja, Joseph [SEP]
500 [CLS] Sunmark Store [SEP] Artificial Intelligence: Structures and Strategies for Complex Problem-Solving [SEP] 0 [SEP]
600 [CLS] DVD Legacy [SEP] Compilers: Principles, Techniques and Tools [SEP] Aho, Alfred V.; Sethi, Ravi; etc. [SEP]
700 [CLS] textbookxdotcom [SEP] Computer Systems Organization AND Architecture [SEP] John D. Carpinell

7100 [CLS] Book Barn Ltd [SEP] AUTHENTICATION: FROM PASSWORDS TO PUBLIC KEYS [SEP] RICHARD E. SMITH [SEP]
7200 [CLS] eCampus.com [SEP] The Cert Guide to System and Network Security Practices [SEP] Allen, Julia H. [SEP]
7300 [CLS] TheBookCom [SEP] Modelling And Analysis Of Security Protocols [SEP] Ryan, Peter [SEP]
7400 [CLS] OPOE-ABE Books [SEP] DIGITAL CERTIFICATES-W/CD [SEP] FEGHHI [SEP]
7500 [CLS] A1Books [SEP] Computers Under Attack: Intruders, Worms and Viruses [SEP] Denning, Peter J. [SEP]
7600 [CLS] Ham Books [SEP] Multimedia Communications: Applications, Networks, Protocols, and Standards [SEP] Halsall, Fred [SEP]
7700 [CLS] textbooksNow [SEP] Telecommunications Essentials [SEP] Goleniewski [SEP]
7800 [CLS] eCampus.com [SEP] Vpns Illustrated: Tunnels, VPNs, and IPsec [SEP] Snader, Jon [SEP]
7900 [CLS] Gunars Store [SEP] Practical BGP [SEP] White Russ et al [SEP]
8000 [CLS] Paperbackshop-US [SEP] Windows Forensics And Incident Recovery [SEP] Carvey, Harlan [SEP]
8100 [CLS] textb

14100 [CLS] brandnewtextbooks [SEP] Principles Of Computer Security: Security+ And Beyond [SEP] Conklin, Art; Davis, Roger; Williams, Dwayne; Cothren, Chuck [SEP]
14200 [CLS] Blackwell Online [SEP] Verilog Computer-based Training Course [SEP] Zainalabedin Navabi [SEP]
14300 [CLS] Collegebooksdirect.com [SEP] Computing Essentials [SEP] O'Leary, Timothy J.; O'Leary, Linda I. [SEP]
14400 [CLS] Browns Books [SEP] Introduction to Help Desk Concepts and Skills [SEP] Sanderson, Susan [SEP]
14500 [CLS] Revaluation Books [SEP] Mcse/mcsa Windows Server 2003 Environment Study Guide Exam 70-290 With Microsoft Windows Server 2003 180-day Trial [SEP] Desai, Anil [SEP]
14600 [CLS] Caiman [SEP] SIMNET CONCEPTS RELEASE 15 D [SEP] Triad, Interactive [SEP]
14700 [CLS] Paperbackshop-US [SEP] Complete Digital Design [SEP] Balch, Mark [SEP]
14800 [CLS] PlumCircle [SEP] Children's Literature and Computer Based Teaching [SEP] Len Unsworth, Angela Thomas, Alyson M. Simpson, Jennifer L Asha [SEP]
14900 [CLS] Bo

20600 [CLS] A1Books [SEP] Web Application Design Handbook : Best Practices for Web-Based Software [SEP] Susan Fowler [SEP]
20700 [CLS] FREE POSTAGE ! @thebookcompany [SEP] Hci Models, Theories And Frameworks : Toward A Multidisciplinary Science [SEP] Carroll, John M. (ed.) [SEP]
20800 [CLS] TheBookCom [SEP] Usability For The Web : Designing Web Sites That Work [SEP] Brinck, Tom [SEP]
20900 [CLS] Hippo Books [SEP] The Usability Engineering Lifecycle: A Practitioner's Handbook for User Interface Design (The Morgan Kaufmann Series in Interactive Technologies) [SEP] Deborah J. Mayhew [SEP]
21000 [CLS] Stratford Books [SEP] Contextual Design, Defining Customer-Centered Systems [SEP] By (author) Beyer, Hugh By (author) Holtzblatt, Karen [SEP]
21100 [CLS] The Book Depository [SEP] Understanding Digital Libraries [SEP] Michael Lesk [SEP]
21200 [CLS] Bobs Books [SEP] Managing Gigabytes [SEP] Ian Witten [SEP]
21300 [CLS] Books Down Under [SEP] OpenVMS System Management Guide [SEP] Baldwin,Lawren

存储中间结果

In [67]:
torch.save( data['encode'] , './DataSet/book/claims_encode.pt' )
data['encode'] = None
data.to_csv( './DataSet/book/claims.txt' , sep='\t' , index=False )

读取中间结果

In [15]:
data = None
data = pd.read_csv( "./DataSet/book/claims.txt" , sep='\t' )
#n_data['source'] = n_data['source'].astype(str)
#n_data['isbn'] = n_data['isbn'].astype(str)
#n_data['name'] = n_data['name'].astype(str)
#n_data['author'] = n_data['author'].astype(str)
data['encode'] = torch.load("./DataSet/book/claims_encode.pt")
GoldenLabel = pd.read_table("./DataSet/book/book_golden.txt" , sep='\t' , header=None , names=['isbn','author'])
SilverLabel = pd.read_table("./DataSet/book/book_silver.txt" , sep='\t' , header=None , names=['isbn','author'])

这里讨巧使用`difflib`来度量字符串相似度，后续可能需要改进，考虑再次引入`bert`

In [88]:
data['label'] = None
data['label'] = data['label'].astype(bool)
data_golden = pd.DataFrame(data.drop(data.index,inplace=False))

for index1,row1 in GoldenLabel.iterrows():
    str1 = row1['author'].lower()
    data_goldenSlice = pd.DataFrame(data[data['isbn']==row1['isbn']])
    for index2,row2 in data_goldenSlice.iterrows():
        str2 = row2['author'].lower()
        data_goldenSlice.loc[index2,'label'] = (difflib.SequenceMatcher(None,str1,str2).quick_ratio()>0.8)
    data_golden = data_golden.append(data_goldenSlice)

存储中间结果

In [89]:
data_golden.reset_index(drop=True,inplace=True)
torch.save( data_golden['encode'] , './DataSet/book/golden/claims_golden_encode.pt' )
data_golden['encode'] = None
data_golden.to_csv( './DataSet/book/golden/claims_golden.txt' , sep='\t' , index=False )

读取中间结果

In [90]:
data_golden = pd.read_csv( './DataSet/book/golden/claims_golden.txt' , sep='\t' )
data_golden['encode'] = torch.load('./DataSet/book/golden/claims_golden_encode.pt')

book_silver.txt数据集有点问题，下一块代码暂且搁置

In [23]:
'''
data['label'] = None
data['label'] = data['label'].astype(bool)
data_silver = pd.DataFrame(data.drop(data.index,inplace=False))

for index1,row1 in SilverLabel.iterrows():
    str1 = row1['author'].lower()
    data_silverSlice = pd.DataFrame(data[data['isbn']==row1['isbn']])
    for index2,row2 in data_silverSlice.iterrows():
        str2 = row2['author'].lower()
        data_silverSlice.loc[index2,'label'] = (difflib.SequenceMatcher(None,str1,str2).quick_ratio()>0.8)
    data_silver = data_silver.append(data_silverSlice)
'''

"\ndata['label'] = None\ndata['label'] = data['label'].astype(bool)\ndata_silver = pd.DataFrame(data.drop(data.index,inplace=False))\n\nfor index1,row1 in SilverLabel.iterrows():\n    str1 = row1['author'].lower()\n    data_silverSlice = pd.DataFrame(data[data['isbn']==row1['isbn']])\n    for index2,row2 in data_silverSlice.iterrows():\n        str2 = row2['author'].lower()\n        data_silverSlice.loc[index2,'label'] = (difflib.SequenceMatcher(None,str1,str2).quick_ratio()>0.8)\n    data_silver = data_silver.append(data_silverSlice)\n"